In [48]:
import character as char
import json
import pandas as pd
import importlib
import re


In [133]:
import random as random
from abc import ABC, abstractmethod
from typing import Optional, Dict
from dataclasses import dataclass
class Dice:
    @staticmethod
    def roll(sides=20, count=1,advantage=None):
        if advantage is None:
            results = [random.randint(1, sides) for _ in range(count)]
            print(f"Individual rolls: {results}")
            total = sum(results)
            print(f"Total: {total}")
            return {"total":total,
                        "dice":results}
        if advantage=="adv":
            if (count!=1 or sides!=20):
                raise ValueError("advantage must be for a one d20 roll")
            else:
                r1 = random.randint(1, sides)
                r2 = random.randint(1, sides)
                print(f"Rolled: {r1} and {r2} -> taking {'highest' if advantage=="adv" else 'lowest'}: {max(r1, r2)}")
                return {"total":max(r1, r2),
                        "dice":[r1,r2]}
        elif advantage=="dis":
            if (count!=1 or sides!=20):
                raise ValueError("advantage must be for a one d20 roll")
            else:
                r1 = random.randint(1, sides)
                r2 = random.randint(1, sides)
                print(f"Rolled: {r1} and {r2} -> taking {'highest' if advantage=="adv" else 'lowest'}: {min(r1, r2)}")
                return {"total":min(r1, r2),
                        "dice":[r1,r2]}
        else:
            raise ValueError("advantage must be one of adv or dis")


class DiceHandler:
    """
    Handles rolling dice with multiple dice specs, modifiers, and additional features.
    Advantage/disadvantage is handled inside the Dice class, so no extra logic needed here.
    """

    def __init__(self):
        pass

    def roll(self, dice_specs, modifiers=0, features=None, advantage=None):
        """
        dice_specs: list of tuples [(sides, count), ...]
        modifiers: int (flat bonuses)
        features: list of callables (rolls, total) -> new rolls, new total
        advantage: passed through to Dice.roll() if needed

        Returns: dict with rolls and final total
        """
        all_rolls = []

        for sides, count in dice_specs:
            # Call Dice.roll() with count, sides, and advantage
            result = Dice.roll(sides=sides, count=count, advantage=advantage)
            all_rolls.extend(result["dice"])

        total = sum(all_rolls)

        
        # Apply any features - these should only affect the dice?
        if features:
            for feature in features:
                all_rolls, total = feature(all_rolls, total)

        # Apply modifiers
        total += modifiers        
        
        # Print debug info
        if len(all_rolls) > 1 or advantage:
            print(f"Final rolls: {all_rolls}")
        print(f"Total after modifiers/features: {total}")

        return {"total": total, "dice": all_rolls}




In [ ]:
# Features that affect dice rolls need to take rolls, total as input and output new rolls and new totals
def halfing_luck(rolls, total):
    # reroll any 1s or 2s
    new_rolls = [r if r > 1 else Dice.roll(sides=20, count=1)["dice"][0] for r in rolls]
    new_total = sum(new_rolls)
    return new_rolls, new_total

In [142]:
DiceHandler().roll(dice_specs=[(20,5)], modifiers=3, features=[halfing_luck], advantage=None)

Individual rolls: [10, 5, 1, 9, 4]
Total: 29
Individual rolls: [8]
Total: 8
Final rolls: [10, 5, 8, 9, 4]
Total after modifiers/features: 39


{'total': 39, 'dice': [10, 5, 8, 9, 4]}

In [113]:
Dice().roll(12,8)


Individual rolls: [5, 10, 8, 9, 5, 3, 3, 9]
Total: 52


{'total': 52, 'dice': [5, 10, 8, 9, 5, 3, 3, 9]}

In [19]:
importlib.reload(char)

<module 'character' from 'c:\\Users\\johnf\\Documents\\githubs\\auto_dnd\\src\\character.py'>

In [22]:
john = char.PCFactory().create_basic(name="John", # str
                     race="Dwarf", # str name of valid race
                     background="Acolyte", # str name of valid background
                     char_class="Bard", # str name of valid class
                     ability_method="roll", # one of [standard, roll, point_buy]
                     ability_score_assignment=["CHA","CON","STR","DEX","INT","WIS"], # ["STR","DEX","CON","INT","WIS","CHA"]
                     ability_score_values=None # list of valid point buy numbers [8,10,11,13,15,8]
                     )

In [27]:
john.ability_scores.modifier("WIS")

0

In [ ]:
# # Get unique official spells

# with open("../data/spells.json") as f:
#     data = json.load(f)

# df = pd.json_normalize(data)

# df_woc = df[df["publisher"]=="Wizards of the Coast"]
# df_woc = df_woc[~df_woc["book"].isin(["Player's Handbook", "Free Basic Rules (2024)", "Free Basic Rules (2014)", "Explorer's Guide to Wildemount (deprecated)",
# "(Deprecated) Rime of the Frostmaiden "])]
# woc_spells = df_woc.drop_duplicates(subset="name", keep="first").reset_index(drop=True)
# woc_spells = woc_spells.drop(columns=["publisher", "properties.Category"])
# woc_spells['properties.School'] = woc_spells['properties.School'].str.capitalize()
# woc_spells['properties.Damage Type'] = woc_spells['properties.Damage Type'].str.capitalize()
# woc_spells = woc_spells.dropna(axis=1, how='all')
# #woc_spells.to_csv("../data/woc_spells.csv")

In [2]:
df = pd.read_csv("../data/woc_monsters.csv")

In [3]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   name             2352 non-null   object
 1   description      2328 non-null   object
 2   Size             2340 non-null   object
 3   Type             2340 non-null   object
 4   Alignment        2215 non-null   object
 5   ChallengeRating  2209 non-null   object
dtypes: object(6)
memory usage: 110.4+ KB


In [68]:
# Parse each description
df["description"] = df["description"].astype("string")

end_kws = ["Actions", "Bonus Actions", "Reactions", "Lair Actions"]
df["traits"] = df["description"].str.extract(rf"({"Traits"}.*?)(?={'|'.join(end_kws)}|$)")

end_kws = ["Spellcasting", "Traits", "Bonus Actions", "Reactions", "Lair Actions"]
end_pattern = "|".join(map(re.escape, end_kws))
start_kw = r"\b(?!Bonus Actions\b|Lair Actions\b|Legendary Actions\b)Actions\b"
pattern = rf"(?s)({start_kw}.*?)(?={end_pattern}|$)"
df["actions"] = df["description"].str.extract(pattern)

end_kws = ["Spellcasting", "Actions", "Traits", "Reactions", "Lair Actions"]
df["bonus_actions"]= df["description"].str.extract(rf"({"Bonus Actions"}.*?)(?={'|'.join(end_kws)}|$)")

end_kws = ["Spellcasting", "Actions", "Bonus Actions", "Traits", "Lair Actions"]
df["reactions"]= df["description"].str.extract(rf"({"Reaction"}.*?)(?={'|'.join(end_kws)}|$)")

end_kws = ["Spellcasting", "Actions", "Bonus Actions", "Reactions", "Traits"]
df["lair_actions"]= df["description"].str.extract(rf"({"Lair Actions"}.*?)(?={'|'.join(end_kws)}|$)")

end_kws = ["Spellcasting", "Actions", "Bonus Actions", "Reactions", "Traits"]
df["legendary_actions"]= df["description"].str.extract(rf"({"Legendary Actions"}.*?)(?={'|'.join(end_kws)}|$)")

end_kws = ["Actions", "Bonus Actions", "Reactions", "Lair Actions"]
df["spellcasting"] = df["description"].str.extract(rf"({"Spellcasting"}.*?)(?={'|'.join(end_kws)}|$)")

end_kws = ["Actions", "Bonus Actions", "Reactions", "Lair Actions","Traits"]
df["short_description"] = df["description"].str.extract(rf"({"^."}.*?)(?={'|'.join(end_kws)}|$)")

In [70]:
df.to_csv("../data/woc_monsters2.csv")

In [ ]:
'Agdon Longscarf, the Most Notorious Brigand in Prismeer This harengon brigand is an insufferable braggart and a daring thief'
' who’s willing to put himself in seemingly precarious situations when he has an audience, confident that his speed and '
'cunning will see him through danger. He leads by example rather than by dictate. Alignment. Chaotic evil. Personality Trait.'
' “Responsibility isn’t really my thing.” Ideal. “What’s yours is mine; it’s only a matter of time.” Bond. “I have a reputation'
'to uphold. I can’t have it sullied by silly concepts like honesty and generosity.” Flaw. “My confidence is bound up in my scarf’s'
' powers. I’m quite the coward without it.” Traits Evasion. If Agdon is subjected to an effect that allows him to make a Dexterity'
' saving throw to take only half damage, he instead takes no damage if he succeeds on the saving throw and only half damage if he'
' fails, provided he isn’t incapacitated. Standing Leap. Agdon’s long jump is up to 20 feet and his high jump is up to 10 feet,'
' with or without a running start. Actions Multiattack. Agdon makes two Branding Iron or Dagger attacks. Branding Iron. '
'Melee Weapon Attack: +7 to hit, reach 5 ft., one target. Hit : 10 (3d6) fire damage, and the target is magically branded. '
'Agdon is invisible to creatures branded in this way. The brand disappears after 24 hours, or it can be removed from a creature '
'or object by any spell that ends a curse. Dagger. Melee or Ranged Weapon Attack: +7 to hit, reach 5 ft. or range 20/60 ft., '
'one target. Hit : 7 (1d4 + 5) piercing damage. Bonus Actions Quick Fingers. Agdon targets one creature within 5 feet of him '
'that he can see and makes a Dexterity (Sleight of Hand) check, with a DC equal to 1 + the target’s passive Wisdom (Perception)'
' score. On a successful check, Agdon pilfers one object weighing 1 pound or less that the target has in its possession but not '
'in its grasp, without the target noticing the theft. Reactions Uncanny Dodge. Agdon halves the damage that he takes from an attack that hits him. He must be able to see the attacker.'

In [84]:
class Spell:
    def __init__(self, data):
        self.name = data["name"]
        self.description = data["description"]
        self.level = data["Level"]
        self.school = data["School"]
        self.components = data["Components"]
        self.dmg_type = data["DamageType"]
        self.cast_time = data["CastingTime"]
        self.range = data["Range"]
        self.save = data["Save"]


In [ ]:
SPELLS = {
    data["name"]: Spell(data)
    for spell_id, data in  df.to_dict(orient="index").items()
}

In [43]:
df = pd.read_csv("../data/woc_backgrounds.csv")

In [46]:
sum([1 for val in ["Fighter", "Fighter", "Fighter", "Rogue"] if val=="Fighter"])

3

In [39]:
df[df["name"]=="Aasimar"].iloc[0].to_dict()

{'name': 'Aasimar',
 'description': 'Whereas tieflings have fiendish blood in their veins, aasimar are the descendants of celestial beings. These folk generally appear as glorious humans with lustrous hair, flawless skin, and piercing eyes. Aasimar often attempt to pass as humans in order to right wrongs and defend goodness on the Material Plane without drawing undue attention to their celestial heritage. They strive to fit into society, although they usually rise to the top, becoming revered leaders and honorable heroes. You might decide to use the aasimar as a counterpoint to the tiefling race. The two races could even be at odds, reflecting some greater conflict between the forces of good and evil in your campaign. Here are our basic goals for the aasimar: Aasimar should make effective clerics and paladins. Aasimar should be to celestials and humans what tieflings are to fiends and humans. Given that aasimar and tieflings are like two sides of the same coin, the tiefling makes a goo

In [40]:
class Race:
    def __init__(self, id):
        df = pd.read_csv("../data/woc_races.csv")
        self.id = id
        if id not in df["name"].values:
            raise ValueError(f"{id} not a valid race.")
        self.racial_data=df[df["name"]==id].iloc[0].to_dict()


    def apply(self, character):
        character.ability_scores.apply_bonuses(self.ability_bonuses)

        for feature in self.features:
            character.features.add(feature)

In [24]:
import random as random
def roll_4d6_drop_lowest():
        result = []
        for _ in range(6):
            rolls = sorted([random.randint(1, 6) for _ in range(4)])
            result.append(sum(rolls[1:]))
         # Returns a list of six scores
        return result

roll_4d6_drop_lowest()

[17, 10, 17, 13, 12, 16]